In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb as dd

# Load the data

df = pd.read_csv('C:/Projects/GitHub/stldata.io/csb_data/merged_csb_data.csv',dtype=str)

In [77]:
#UPDATE DATE TYPES
df['DATETIMECLOSED'] = pd.to_datetime(df['DATETIMECLOSED'])
df['DATEINVTDONE'] = pd.to_datetime(df['DATETIMECLOSED'])
df['DATETIMEINIT'] = pd.to_datetime(df['DATETIMEINIT'])
df['DATECANCELLED'] = pd.to_datetime(df['DATECANCELLED']) 

#Create Speed to Resolution Time Delta, convert to float and round to 2 decimal places
df['SpeedToResolution'] = df['DATETIMECLOSED'] - df['DATETIMEINIT']
df['SpeedToResolution'] = df['SpeedToResolution'].dt.total_seconds()/60/60/24
df['SpeedToResolution'] = df['SpeedToResolution'].round(2)

df['DATETIMECLOSED'] = pd.to_datetime(df['DATETIMECLOSED'] , errors='coerce' ).round('D')
df['DATEINVTDONE'] = pd.to_datetime(df['DATEINVTDONE'], errors='coerce').round('D')
df['DATETIMEINIT'] = pd.to_datetime(df['DATETIMEINIT'], errors='coerce').round('D')
df['DATECANCELLED'] = pd.to_datetime(df['DATECANCELLED'], errors='coerce').round('D')



#Drop columns that are not needed
df.drop(['EXPLANATION'] , axis=1, inplace=True)
df.drop(['SRX'] , axis=1, inplace=True)
df.drop(['SRY'] , axis=1, inplace=True)
df.drop(['PROBADDRESS'] , axis=1, inplace=True)
df.drop(['CITY'] , axis=1, inplace=True)
df.drop(['CALLERTYPE'] , axis=1, inplace=True)
df.drop(['PROBLEMSID'] , axis=1, inplace=True)
df.drop(['PROBZIP'] , axis=1, inplace=True)
df.drop(['PARENT_ID'] , axis=1, inplace=True)
df.drop(['NEIGHBORHOOD'] , axis=1, inplace=True)
df.drop(['GRANDPARENT_ID'] , axis=1, inplace=True)
df.drop(['PROBADDTYPE'] , axis=1, inplace=True)


#identify and overwrite values in the datecancelled, dateinvtdone, datetimeclosed, datetimeinit columns if they are greater than the current date, replace the value with null
df['DATECANCELLED'] = np.where(df['DATECANCELLED'] > pd.to_datetime('today'), pd.NaT, df['DATECANCELLED'])
df['DATEINVTDONE'] = np.where(df['DATEINVTDONE'] > pd.to_datetime('today'), pd.NaT, df['DATEINVTDONE'])
df['DATETIMECLOSED'] = np.where(df['DATETIMECLOSED'] > pd.to_datetime('today'), pd.NaT, df['DATETIMECLOSED'])
df['DATETIMEINIT'] = np.where(df['DATETIMEINIT'] > pd.to_datetime('today'), pd.NaT, df['DATETIMEINIT'])

df.replace(np.nan, None, inplace=True)

#convert datecancelled, dateinvtdone, datetimeclosed, datetimeinit columns from nanoseconds to microseconds
df['DATECANCELLED'] =pd.to_numeric((df['DATECANCELLED']/1000000), downcast='integer').round(0)
df['DATEINVTDONE'] = pd.to_numeric((df['DATEINVTDONE']/1000000), downcast='integer').round(0)
df['DATETIMECLOSED'] = pd.to_numeric((df['DATETIMECLOSED']/1000000), downcast='integer').round(0)
df['DATETIMEINIT'] = pd.to_numeric((df['DATETIMEINIT']/1000000), downcast='integer').round(0)




#CONVERT df['WARD'] TO INT, Replacing any non-numeric values with nulls
df['WARD'] = pd.to_numeric(df['WARD'], errors='coerce', downcast='integer')
df['WARD'] = (df['WARD'].fillna(0)).round(0).astype(int)

#10% of the rows are duplicates, so we will remove them
df.drop_duplicates(inplace=True)

#label remaining duplicates
df['is_duplicated'] = df.duplicated(subset='REQUESTID', keep=False)

df.head()


,DATECANCELLED,DATEINVTDONE,DATETIMECLOSED,DATETIMEINIT,DESCRIPTION,GRANDPARENT_NODE,GROUP,PARENT_NODE,PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE,PRJCOMPLETEDATE,PROBLEMCODE,PUBLICRESOLUTION,REQUESTID,STATUS,SUBMITTO,WARD,SpeedToResolution,is_duplicated
0,NaN,1483228800000,1483228800000,1483228800000,Misc-Street Div,Citizen Requests,Miscellaneous,Miscellaneous,Misc request to Street Division,2017-01-20 00:00:00.0,Misc-Street Div,work completed,925943,CLOSED,"STREET DIVISION, 514",14,0.03,False
1,NaN,1483401600000,1483401600000,1483228800000,Whole block lights out,Citizen Requests,Street Lights,Street Lights,Whole block of lights out,2017-01-04 00:00:00.0,Whole Blk Lights Out,work completed,925944,CLOSED,"TRAFFIC DIVISION, 511",11,2.18,False
2,NaN,1485907200000,1485907200000,1483228800000,Check Welfare,Animals,Animals,Animals and Pets,Check Welfare,2017-01-23 00:00:00.0,Check Welfare,verbal advice given,925945,CLOSED,"ANIMAL CARE,",1,30.24,False
3,NaN,1484006400000,1484006400000,1483228800000,Vehicle Towing,Citizen Requests,Vehicles,Vehicles,"Derelict, vandalized, inoperable, or abandoned...",2017-01-10 00:00:00.0,Vehicle Towing,vehicle gone upon reinspection,925946,CLOSED,"STREET DEPT, 510",4,8.05,False
4,NaN,1485302400000,1485302400000,1483228800000,Stump Removal,Citizen Requests,Trees,Trees,Request for removal of city tree stump,2017-01-23 00:00:00.0,Stump Removal,vehicle gone upon reinspection,925947,CLOSED,"FORESTRY,",9,23.85,False


In [78]:
df.to_parquet('C:/Projects/GitHub/stldata.io/csb_data/merged_csb_df_pq.parquet',index=False, )

In [79]:
#identify how many request ids are duplicates
print(f"{df['REQUESTID'].nunique()} unique request ids with {df['REQUESTID'].count()} total rows")


877899 unique request ids with 877914 total rows


In [80]:
#identify which request ids are duplicated

df_duplicated = df[df['is_duplicated'] == True]
df_duplicated.sort_values(by='REQUESTID', inplace=True)
df_duplicated.head(10)

c:\Users\legis\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,DATECANCELLED,DATEINVTDONE,DATETIMECLOSED,DATETIMEINIT,DESCRIPTION,GRANDPARENT_NODE,GROUP,PARENT_NODE,PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE,PRJCOMPLETEDATE,PROBLEMCODE,PUBLICRESOLUTION,REQUESTID,STATUS,SUBMITTO,WARD,SpeedToResolution,is_duplicated
101597,1.511827e+12,1516060800000,1516060800000,1511827200000,Misc-Street Div,Citizen Requests,Miscellaneous,Miscellaneous,Misc request to Street Division,2017-12-18 00:00:00.0,Misc-Street Div,work completed,1040592,CLOSED,"STREET DIVISION, 514",12,48.98,True
101598,1.511827e+12,1516060800000,1516060800000,1511827200000,Misc-Street Div,Citizen Requests,Miscellaneous,Miscellaneous,Misc request to Street Division,2017-12-18 00:00:00.0,Misc-Street Div,referred to street maintenance for repair work...,1040592,CLOSED,"STREET DIVISION, 514",12,48.98,True
260686,1.558310e+12,1558396800000,1558396800000,1558310400000,Stray Dog At Large,Animals,Animals,Animals and Pets,Report stray dog(s) roaming at large,2019-06-07 00:00:00.0,Stray Dog At Large,apprehended animal,1221836,CLOSED,"ANIMAL CARE,",13,1.08,True
260688,1.558310e+12,1558396800000,1558396800000,1558310400000,Stray Dog At Large,Animals,Animals,Animals and Pets,Report stray dog(s) roaming at large,2019-06-07 00:00:00.0,Stray Dog At Large,apprehended animal,1221836,CLOSED,"ANIMAL CARE,",3,1.08,True
277873,1.565827e+12,1564790400000,1564790400000,1562544000000,Mosquito Spraying,Animals,Animals,Pests and Rodents,Request for mosquito spraying,2019-07-17 00:00:00.0,Mosquito Spraying,treated with adulticide,1241288,CLOSED,"VECTOR CNTRL,",5,25.22,True
277875,1.565827e+12,1564790400000,1564790400000,1562544000000,Mosquito Spraying,Animals,Animals,Pests and Rodents,Request for mosquito spraying,2019-07-17 00:00:00.0,Mosquito Spraying,treated with adulticide,1241288,CLOSED,"VECTOR CNTRL,",9,25.22,True
281892,1.563494e+12,1564099200000,1564099200000,1563408000000,Inspect City Tree,Citizen Requests,Trees,Trees,Inspect city tree,2019-08-07 00:00:00.0,Inspect City Tree,tree scheduled for trimming,1245994,Pending,"FORESTRY,",5,7.88,True
281894,1.563494e+12,1567728000000,1567728000000,1563408000000,Inspect City Tree,Citizen Requests,Trees,Trees,Inspect city tree,2019-08-07 00:00:00.0,Inspect City Tree,tree scheduled for trimming,1245994,Pending,"FORESTRY,",5,49.25,True
362664,1.589501e+12,1591747200000,1591747200000,1589414400000,Misc-License Coll,Citizen Requests,Miscellaneous,Miscellaneous,Container is damaged.,2020-05-28 10:28:38.0,Misc-License Coll,"Request investigated, please call for details",1339477,NEW,"LICENSE COLL,",5,26.10,True
362666,1.589501e+12,1589500800000,1589500800000,1589414400000,Misc-License Coll,Citizen Requests,Miscellaneous,Miscellaneous,Container is turned around,2020-05-28 10:28:38.0,Misc-License Coll,"Request investigated, please call for details",1339477,NEW,"LICENSE COLL,",5,0.13,True


In [81]:
#identify the max date in the datecancelled, dateinvtdone, datetimeclosed, datetimeinit columns
print(f"Max DateCancelled: {df['DATECANCELLED'].max()}")    
print(f"Max DateInvtdone: {df['DATEINVTDONE'].max()}")
print(f"Max DateTimeClosed: {df['DATETIMECLOSED'].max()}")
print(f"Max DateTimeInit: {df['DATETIMEINIT'].max()}")
print(f"Max SpeedToResolution: {df['SpeedToResolution'].max()}")

Max DateCancelled: 1742256000000.0
Max DateInvtdone: 1742256000000
Max DateTimeClosed: 1742256000000
Max DateTimeInit: 1742256000000
Max SpeedToResolution: 3283.39


In [82]:
#identify the min date in the datecancelled, dateinvtdone, datetimeclosed, datetimeinit columns
print(f"Min DateCancelled: {df['DATECANCELLED'].min()}")    
print(f"Min DateInvtdone: {df['DATEINVTDONE'].min()}")
print(f"Min DateTimeClosed: {df['DATETIMECLOSED'].min()}")
print(f"Min DateTimeInit: {df['DATETIMEINIT'].min()}")
print(f"Min SpeedToResolution: {df['SpeedToResolution'].min()}")

Min DateCancelled: 1483401600000.0
Min DateInvtdone: 1483228800000
Min DateTimeClosed: 1483228800000
Min DateTimeInit: 1483228800000
Min SpeedToResolution: -274.56
